In [16]:
import pandas as pd

# Creamos un dataset pequeño de ejemplo
data = [
    ("¡Me encanta este producto, es genial!", 1),  # Positivo
    ("El servicio fue pésimo y lento.", 0),        # Negativo
    ("Llegó a tiempo y funciona bien.", 1),        # Positivo
    ("No me gustó, mala calidad.", 0),             # Negativo
    ("Estoy muy feliz con la compra.", 1)          # Positivo
]

df = pd.DataFrame(data, columns=['texto', 'sentimiento'])
print(df)

                                   texto  sentimiento
0  ¡Me encanta este producto, es genial!            1
1        El servicio fue pésimo y lento.            0
2        Llegó a tiempo y funciona bien.            1
3             No me gustó, mala calidad.            0
4         Estoy muy feliz con la compra.            1


In [17]:
import re

texto_original = "¡Me encanta este producto, es genial!"

# 1. Limpieza (Quitar signos y minúsculas)
texto_limpio = re.sub(r'[^\w\s]', '', texto_original).lower()

# 2. Tokenización (Dividir en palabras)
tokens = texto_limpio.split()

# 3. Stopwords (Quitar palabras vacías manualmente para el ejemplo)
stopwords = {'me', 'este', 'es', 'el', 'la', 'y'}
tokens_filtrados = [w for w in tokens if w not in stopwords]

print(f"Original: {texto_original}")
print(f"Procesado: {tokens_filtrados}")

Original: ¡Me encanta este producto, es genial!
Procesado: ['encanta', 'producto', 'genial']


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
# Transformamos nuestros textos a números
matriz_bow = vectorizer.fit_transform(df['texto'])

# Mostramos el vocabulario aprendido y el vector de la primera frase
print("Vocabulario:", vectorizer.get_feature_names_out())
print("Vector Frase 1:", matriz_bow[0].toarray())

Vocabulario: ['bien' 'calidad' 'compra' 'con' 'el' 'encanta' 'es' 'este' 'estoy'
 'feliz' 'fue' 'funciona' 'genial' 'gustó' 'la' 'lento' 'llegó' 'mala'
 'me' 'muy' 'no' 'producto' 'pésimo' 'servicio' 'tiempo']
Vector Frase 1: [[0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0]]


In [19]:
import gensim.downloader as api

# Descargamos un modelo pre-entrenado (ej. Twitter)
# Esto convierte palabras en coordenadas matemáticas
modelo_w2v = api.load("glove-twitter-25")

# Verificamos si la máquina entiende la relación
vector_rey = modelo_w2v['king']
vector_reina = modelo_w2v['queen']

# La magia: Rey - Hombre + Mujer = ¿Reina?
resultado = modelo_w2v.most_similar(positive=['woman', 'king'], negative=['man'])
print(resultado) # Debería decir 'queen'

[('meets', 0.8841923475265503), ('prince', 0.832163393497467), ('queen', 0.8257461190223694), ('’s', 0.8174097537994385), ('crow', 0.813499391078949), ('hunter', 0.8131037950515747), ('father', 0.8115833401679993), ('soldier', 0.81113600730896), ('mercy', 0.8082392811775208), ('hero', 0.8082262277603149)]


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Vectorizamos con TF-IDF (Más avanzado que Bag of Words)
tfidf = TfidfVectorizer(stop_words=['el', 'la', 'y', 'de'])
X = tfidf.fit_transform(df['texto'])
y = df['sentimiento']

# 2. Entrenamos el modelo (Regresión Logística)
modelo = LogisticRegression()
modelo.fit(X, y)

print("¡Modelo entrenado exitosamente!")

¡Modelo entrenado exitosamente!


In [30]:
# Nuevo comentario que llega al sistema
nuevo_comentario = ["El servicio fue pésimo y lento"]

# 1. Convertimos el texto a números (usando el MISMO transformador)
nuevo_vector = tfidf.transform(nuevo_comentario)

# 2. El modelo predice
prediccion = modelo.predict(nuevo_vector)
probabilidad = modelo.predict_proba(nuevo_vector)

if prediccion[0] == 1:
    print("Resultado: 😀 Positivo")
else:
    print("Resultado: 😡 Negativo")
    
print(f"Confianza: {probabilidad.max():.2f}")

Resultado: 😡 Negativo
Confianza: 0.52
